## Install necessary packages

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()
exit()
import torch
!pip uninstall torch -y
!conda install -c conda-forge pymatgen rdkit openbabel -y
!pip install torch==1.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
!pip install torch_geometric
exit()
!pip install torchmetrics tqdm transformers wandb apex curtsies fairscale imageio joblib Pillow pytorch_lightning PyYAML ray scikit-learn ipykernel docopt
!pip install git+https://github.com/tensorflow/docs

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...
conda 23.1.0


'2.0.1+cu118'

## Clone GHP-MOFAssembly Github repository

In [ ]:
!git clone https://github.com/hyunp2/ghp_mof.git

## Generate new MOF structures
In this example, we generate a total of 4000 Cu paddlewheel MOFs at four catenation levels (cat0/cat1/cat2/cat3 1000 each)

In [39]:
!cd ghp_mof && bash generate_mofs.sh

Step 8 - Removing structures that cannot be read by pymatgen
------------------------------------------------------------
Number of CPUs: 1
/usr/local/lib/python3.10/site-packages/pymatgen/io/cif.py:1134: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
Step 9 - Preparing necessarly files for making predictions
----------------------------------------------------------
copied atom_init.json ...
generated id_prop.csv ...


## Predict the CO2 capacities of generated MOF structures using the modified version of CGCNN model

In [ ]:
!cd ghp_mof && python -m main \
--which_mode infer \
--dataset cifdata \
--backbone cgcnn \
--batch_size 2 \
--load_ckpt_path models \
--name cgcnn_pub_hmof_0.1 \
--gpu \
--train_frac 1 \
--data_dir_crystal MOFs/MOFs_all \
--ensemble_names cgcnn_pub_hmof_0.1 cgcnn_pub_hmof_0.1_dgx cgcnn_pub_hmof_0.1_v2

Truncate_above is False...
0-th core is parsed!
Root dir chosen is MOFs/MOFs_all
Truncate_above is False...
mean None std None
Loaded a model from rank 0!
Loaded a model from rank 0!
Loaded a model from rank 0!


In [43]:
# model predictions
import pandas as pd
pd.read_csv("ghp_mof/publication_figures/ensemble_property_prediction.csv", index_col=0)

,name,pred,std,real
0,hMOF-1.cif,1.703976,0.242664,1.806490
1,hMOF-2.cif,1.058071,0.309409,0.854371
2,hMOF-0.cif,0.208278,0.009849,0.225851


### Optional: Visualization of linker generation process using DiffLinker

In [4]:
# sample difflinker
!cd ghp_mof/DiffLinker && git pull && python -W ignore difflinker_sample_and_analyze.py \
--fragments geom/datasets/geom_multifrag_test_frag.sdf \
--model checkpoints/geom_difflinker.ckpt \
--linker_size checkpoints/geom_size_gnn.ckpt \
--output geom/output \
--samples_dir geom/samples

Already up to date.
Will generate linkers with sampled numbers of atoms
Lightning automatically upgraded your loaded checkpoint from v1.6.3 to v2.0.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file checkpoints/geom_size_gnn.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.6.3 to v2.0.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file checkpoints/geom_difflinker.ckpt`
  0% 0/1 [00:00<?, ?it/s]Sampling only the FINAL output...
Saved generated molecules in .xyz and .sdf format in directory geom/output
Sampling ALL trajectories...
torch.Size([5, 32, 9]) torch.Size([5, 32, 9])
Out of 5 molecules, 0 are stable per B.O. analysis!
Saved generated molecules with B.O in .sdf format in directory geom/output and animation in geom/samples
100% 1/1 [02:22<00:00, 142.14s/it]


In [5]:
import os, base64, tempfile
from openbabel.pybel import readfile
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

workdir = "ghp_mof/DiffLinker"
samples_dir = os.path.join(workdir, "geom/samples")
output_dir = os.path.join(workdir, "geom/output")
mol_ids = [int(x.replace("mol_", "")) for x in os.listdir(samples_dir) if os.path.isdir(os.path.join(samples_dir, x))]
for mol_id in mol_ids:
    mol = "mol_" + str(mol_id)
    gif_path = os.path.join(samples_dir, mol, "output.gif")
    print(gif_path)
    b64 = base64.b64encode(open(gif_path,'rb').read()).decode('ascii')
    display(HTML(f'<img src="data:image/gif;base64,{b64}" />'))

    xyz = "output_" + str(mol_id) + "_geom_multifrag_test_frag_.xyz"
    print(os.path.join(output_dir, xyz))
    mol = list(readfile("xyz", os.path.join(output_dir, xyz)))[-1]

    mol.write("sdf", "tmp.sdf", overwrite=True)
    rdmol = next(iter(Chem.SDMolSupplier("tmp.sdf")))
    display(rdmol)
    if os.path.exists("tmp.sdf"):
        os.remove("tmp.sdf")
    print("\n")

ghp_mof/DiffLinker/geom/samples/mol_4/output.gif


NameError: ignored